## Thuật toán sử dụng phương pháp hàm ngược để giải bài toán nội suy 

In [15]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_and_validate_inverse_data(file_x, file_y):
    """
    Tải dữ liệu từ file và kiểm tra các điều kiện cho nội suy ngược.
    """
    try:
        x_coords = np.loadtxt(file_x, ndmin=1).astype(float)
        y_coords = np.loadtxt(file_y, ndmin=1).astype(float)
    except (FileNotFoundError, ValueError):
        raise IOError("LỖI: Không tìm thấy file hoặc file chứa dữ liệu không hợp lệ.")
    
    # Kiểm tra số lượng điểm
    if len(x_coords) != len(y_coords):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
        
    # KIỂM TRA QUAN TRỌNG: Các giá trị y không được trùng lặp
    # vì chúng sẽ là các mốc nội suy mới.
    if len(np.unique(y_coords)) != len(y_coords):
        raise ValueError("LỖI: Các giá trị y (trong FileY.txt) bị trùng lặp. "
                         "Không thể dùng làm mốc nội suy cho hàm ngược.")
                         
    return x_coords, y_coords

# --- Chương trình chính cho Ô 1 ---
try:
    x_coords, y_coords = load_and_validate_inverse_data('FileX.txt', 'FileY.txt')
    print("Dữ liệu hợp lệ và đã được đọc thành công!")
    print("-" * 50)
    print("Các mốc x ban đầu:", x_coords.tolist())
    print("Các giá trị y ban đầu:", y_coords.tolist())

except (IOError, ValueError) as e:
    print(str(e))

Dữ liệu hợp lệ và đã được đọc thành công!
--------------------------------------------------
Các mốc x ban đầu: [1.118, 1.236, 1.354, 1.472, 1.59, 1.708, 1.826]
Các giá trị y ban đầu: [1.5143, 1.9576, 2.3866, 2.7958, 3.1802, 3.5347, 3.855]


In [16]:
# Giả định các biến x_coords, y_coords, và inverse_div_diff_table đã tồn tại

# --- Hiển thị bảng với cột Y đứng đầu ---

# 1. Tạo DataFrame từ kết quả tính toán
#    inverse_div_diff_table[i, 0] chứa các giá trị x_i
#    inverse_div_diff_table[i, 1] chứa các giá trị TSP1
#    ...
inv_df_display = pd.DataFrame(inverse_div_diff_table)

# 2. Đặt tên cột cho phù hợp
#    Cột 0 là 'x', cột 1 là 'TSP1', ...
col_names = ['x'] + [f'TSP{i}' for i in range(1, len(x_coords))]
inv_df_display.columns = col_names

# 3. Chèn cột 'y' (các mốc) vào vị trí đầu tiên
inv_df_display.insert(0, 'y', y_coords) # Chèn cột y vào cột 0

print("--- Bảng Tỷ sai phân Ngược (Cột y đầu tiên) ---")
# Sử dụng style để hiển thị số nhỏ và NaN (không làm tròn)
display(inv_df_display.style.format("{:g}", na_rep=""))

# (Tùy chọn) In lại các hệ số Newton cho P(y)
inverse_newton_coeffs = inverse_div_diff_table[0]
print("\n--- Các hệ số Newton cho đa thức P(y) (Hàng đầu tiên của các cột TSP) ---")
print(inverse_newton_coeffs)

--- Bảng Tỷ sai phân Ngược (Cột y đầu tiên) ---


,y,x,TSP1,TSP2,TSP3,TSP4,TSP5,TSP6
0,1.5143,1.118,0.266185,0.0101718,0.00445307,0.00104102,0.000713106,0.000139781
1,1.9576,1.236,0.275058,0.0158784,0.0061873,0.00248178,0.00104029,0
2,2.3866,1.354,0.288368,0.023443,0.0101013,0.00445562,0,0
3,2.7958,1.472,0.306972,0.0350403,0.016644,0,0,0
4,3.1802,1.59,0.332863,0.0526696,0,0,0,0
5,3.5347,1.708,0.368405,0,0,0,0,0
6,3.855,1.826,0,0,0,0,0,0



--- Các hệ số Newton cho đa thức P(y) (Hàng đầu tiên của các cột TSP) ---
[1.11800000e+00 2.66185427e-01 1.01717845e-02 4.45307168e-03
 1.04101705e-03 7.13106169e-04 1.39780682e-04]


In [23]:
import numpy as np
import pandas as pd
from numpy.polynomial import polynomial as P

# Giả định x_coords và y_coords đã tồn tại từ các ô trước.
# Ví dụ dữ liệu (lấy từ ảnh a16a7e.png) để bạn có thể chạy thử nghiệm:
# x_coords = np.array([1.118, 1.236, 1.354, 1.472, 1.59, 1.708, 1.826])
# y_coords = np.array([1.5143, 1.9576, 2.3866, 2.7958, 3.1802, 3.5347, 3.855])

def display_lower_tri_div_diff(x_values, y_values):
    """
    Hàm này tạo, hiển thị bảng tỷ sai phân NGƯỢC dạng tam giác dưới
    và tô màu các giá trị trên đường chéo chính.
    """
    
    n = len(x_values)
    
    # --- 1. Tính toán Bảng Tỷ sai phân Ngược ---
    #    Tạo bảng ma trận (tam giác trên) để tính toán
    #    Bảng này tính f[yi, yi+1, ...] = (f[yi+1,...] - f[yi,...]) / (yi+k - yi)
    #    (Sử dụng y_values làm mốc, x_values làm giá trị)
    calc_table = np.full((n, n), np.nan)
    calc_table[:, 0] = x_values # Cột đầu tiên là x

    for j in range(1, n): # Cột (bậc tỷ sai phân)
        for i in range(n - j): # Hàng
            numerator = calc_table[i+1, j-1] - calc_table[i, j-1]
            denominator = y_values[i+j] - y_values[i]
            calc_table[i, j] = numerator / denominator
            
    # --- 2. Tạo DataFrame Hiển thị (Dạng Tam giác Dưới) ---
    #    Tạo bảng rỗng (NaN)
    display_table = np.full((n, n), np.nan)
    
    # Lặp và điền các giá trị vào đúng vị trí tam giác dưới
    for j in range(n): # Cột (bậc TSP)
        for i in range(j, n): # Hàng
            # Lấy giá trị từ bảng tính (hàng i-j, cột j)
            # Ví dụ: (hàng 2, cột 1) của bảng dưới = (hàng 1, cột 1) của bảng trên
            display_table[i, j] = calc_table[i-j, j]
            
    # Tạo DataFrame từ bảng tam giác dưới
    headers = [f'TSP{i}' for i in range(n)]
    df_display = pd.DataFrame(display_table, columns=headers)
    
    # Đổi tên cột đầu tiên thành 'x'
    df_display.rename(columns={'TSP0': 'x'}, inplace=True)
    
    # Thêm cột 'y' vào đầu
    df_display.insert(0, 'y', y_values)

    # --- 3. Hàm tô màu đường chéo ---
    def highlight_diagonal(data):
        style = 'background-color: yellow; font-weight: bold; color: black;'
        style_df = pd.DataFrame('', index=data.index, columns=data.columns)
        
        # Lặp và tô màu đường chéo
        # Bỏ qua cột 'y' (cột 0), bắt đầu từ cột 'x' (cột 1)
        for i in range(n):
            col_idx_in_df = i + 1 # Cột 'x' là 1, 'TSP1' là 2, ...
            row_idx_in_df = i
            
            if col_idx_in_df < len(data.columns):
                val = data.iloc[row_idx_in_df, col_idx_in_df]
                if not pd.isna(val): # Chỉ tô màu ô có giá trị
                    style_df.iloc[row_idx_in_df, col_idx_in_df] = style
        return style_df

    # --- 4. Áp dụng style và hiển thị ---
    print("--- Bảng Tỷ sai phân Ngược (Dạng tam giác dưới, tô màu đường chéo) ---")
    
    numeric_cols = df_display.select_dtypes(include=np.number).columns
    styled_df = df_display.style.apply(highlight_diagonal, axis=None).format(
        "{:g}", 
        subset=numeric_cols, 
        na_rep="" # Hiển thị ô trống thay vì 'NaN'
    )
    
    display(styled_df)
    
    # Trả về các hệ số Newton (Hàng đầu tiên của bảng tính toán)
    return calc_table[0, :]

# --- Thực thi hàm ---
try:
    # Chạy hàm để hiển thị bảng
    inverse_newton_coeffs = display_lower_tri_div_diff(x_coords, y_coords)
    
    # In các hệ số đã được tô màu
    print("\n--- Các hệ số Newton của đa thức P(y) (giá trị trên đường chéo, hàng đầu của bảng tính) ---")
    print(inverse_newton_coeffs)
    
except NameError:
    print("LỖI: Các biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bảng Tỷ sai phân Ngược (Dạng tam giác dưới, tô màu đường chéo) ---


,y,x,TSP1,TSP2,TSP3,TSP4,TSP5,TSP6
0,1.5143,1.118,,,,,,
1,1.9576,1.236,0.266185,,,,,
2,2.3866,1.354,0.275058,0.0101718,,,,
3,2.7958,1.472,0.288368,0.0158784,0.00445307,,,
4,3.1802,1.59,0.306972,0.023443,0.0061873,0.00104102,,
5,3.5347,1.708,0.332863,0.0350403,0.0101013,0.00248178,0.000713106,
6,3.855,1.826,0.368405,0.0526696,0.016644,0.00445562,0.00104029,0.000139781



--- Các hệ số Newton của đa thức P(y) (giá trị trên đường chéo, hàng đầu của bảng tính) ---
[1.11800000e+00 2.66185427e-01 1.01717845e-02 4.45307168e-03
 1.04101705e-03 7.13106169e-04 1.39780682e-04]
